# Workflow Validation, Extension, and Tracking

This notebook demonstrates how to validate, extend, and track the simulation workflow with new data sections, including automated tests, example extensions, user-facing error handling, and changelog management.

---

## 1. Automated Tests for Data Section Parsing

The following cells use `pytest`-style assertions to ensure all new data sections in `forcefield_params.json` are present and correctly formatted. These tests help guarantee that the workflow can parse and use the new parameters for dihedrals, beads, ions, PFAS, antibiotics, and microplastics.

In [ ]:
import json
import os

def load_forcefield():
    with open('../data/forcefield_params.json', 'r') as f:
        return json.load(f)

ff = load_forcefield()

# Test dihedral_types
assert 'dihedral_types' in ff, "Missing 'dihedral_types' section."
for key in ["O_double-C_carb-O_single", "C-O_epoxy-C", "CT-N-CT", "O_S-O_S-S"]:
    assert key in ff['dihedral_types'], f"Missing dihedral: {key}"

# Test coarse_grained_beads
assert 'coarse_grained_beads' in ff, "Missing 'coarse_grained_beads' section."
for bead in ["PL", "RV"]:
    assert bead in ff['coarse_grained_beads'], f"Missing bead: {bead}"

# Test regeneration_chemistry
assert 'regeneration_chemistry' in ff, "Missing 'regeneration_chemistry' section."
assert 'ClO' in ff['regeneration_chemistry'], "Missing ClO in regeneration_chemistry."
assert 'bonds' in ff['regeneration_chemistry'], "Missing bonds in regeneration_chemistry."
assert 'O-ClO' in ff['regeneration_chemistry']['bonds'], "Missing O-ClO bond."

# Test pfas_cross_terms
assert 'pfas_cross_terms' in ff, "Missing 'pfas_cross_terms' section."
for key in ["PFOA-C_carb", "PFOS-O_double", "PFBS-CT"]:
    assert key in ff['pfas_cross_terms'], f"Missing PFAS cross-term: {key}"

# Test antibiotic_parameters
assert 'antibiotic_parameters' in ff, "Missing 'antibiotic_parameters' section."
for key in ["Cipro_C-N", "Cipro_N-C-C", "Tetra_O-C-O"]:
    assert key in ff['antibiotic_parameters'], f"Missing antibiotic parameter: {key}"

# Test microplastic_hybrid
assert 'microplastic_hybrid' in ff, "Missing 'microplastic_hybrid' section."
for key in ["PET", "PS"]:
    assert key in ff['microplastic_hybrid'], f"Missing microplastic: {key}"

print("All new data sections are present and correctly formatted.")

---

## 2. Example: Adding New Materials

This section demonstrates how to add a new material (e.g., a microplastic) to the workflow by updating `forcefield_params.json` with the required parameters. This ensures the new material is available for simulation and analysis.

In [ ]:
# Add a new microplastic (Polypropylene, PP) to forcefield_params.json
import json

with open('../data/forcefield_params.json', 'r') as f:
    ff = json.load(f)

# Add Polypropylene (PP)
ff.setdefault('microplastic_hybrid', {})['PP'] = {
    "sigma": 4.10,
    "epsilon": 0.09,
    "hydration_energy": -13.5
}

with open('../data/forcefield_params.json', 'w') as f:
    json.dump(ff, f, indent=2)

print("Added Polypropylene (PP) to microplastic_hybrid section.")

---

## 3. Example: Running Custom Scenarios

This section shows how to run a custom scenario using the workflow, such as simulating a membrane with a new background ion or microplastic. You can specify scenario parameters and interpret the results using the workflow's main functions.

In [ ]:
# Example: Run a custom scenario with a new background ion (e.g., HCO3)
from src.simulate_chemistry import ChemicalSimulationEngine

# Add a new scenario: simulate HCO3- removal with a hybrid membrane
environment = ChemicalSimulationEngine()
custom_contaminants = ['HCO3']
custom_concentrations = {'HCO3': 50.0}

results = environment.simulate_contaminant_removal(
    membrane_type='hybrid',
    contaminants=custom_contaminants,
    initial_concentrations=custom_concentrations,
    reaction_time=120
)

print("Custom scenario results:")
print(results)